# Equipo 2
## Miembros:
### Jorge Arturo Torres Cruz - A01176590
### Juan Manuel Pérez Font - A00819815

## Recolección de datos

In [1]:
# Para uso en colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# ==== FOR REFERENCE =====
# ['wines.txt', 'bubble_gums.txt', 'dumplings.txt', 'pizza_urls.txt', 'sandwich.txt']
# =========== ============
url_files = ['dumplings.txt', 'pizza_urls.txt', 'sandwich.txt']

# ==== FOR REFERENCE =====
# ['wine','bubble_gum', 'dumpling', 'pizza', 'sandwich']
# ========================
categories = ['dumpling', 'pizza', 'sandwich'] # wine bubble_gum

data_directory = './drive/My Drive/data'

In [2]:
# Iteramos por cada uno de los archivos y obtenemos los URLs de las imagenes.
print("Reading files containing images urls")
urls = []
for file_name in url_files:
  with open(file_name) as f:
    content = f.readlines()
    content = [url.strip() for url in content]
    print("Reading {} image urls ({})".format(len(content), file_name))
    urls.append(content)

Reading files containing images urls
Reading 1272 image urls (dumplings.txt)
Reading 1215 image urls (pizza_urls.txt)
Reading 1229 image urls (sandwich.txt)


In [0]:
# Utilizaremos urllib para descargar las imagenes utilizando URLs obtenidos de ImageNet
import urllib.request
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
import uuid
from itertools import repeat
import socket
socket.setdefaulttimeout(10)

In [10]:
def download_from_url(category_url):
  category, url = category_url
  print(f'Downloading {url} for category {category}')
  try:
    urllib.request.urlretrieve(url, f'{data_directory}/{category}/{uuid.uuid4()}.jpg')
    return url
  except Exception as e:
    print('Error')
    return f'Error: {e}'

def download_category_from_url(category, urls):
  try:
    print(f'Creating directory to store {category} images')
    os.mkdir(f'{data_directory}/{category}')
  except FileExistsError:
    print(f'{data_directory}/{category} directory exists, continuing...')
  except Exception as e:
    print(e)
  else:
    print(f'Succesfully created {category}/ directory')
  results = None
  print(f'Downloading images for category {category}')
  with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(download_from_url, zip(repeat(category), urls), timeout=2)
  print(f'All images downloaded for category {category}')

try:
  os.mkdir(data_directory)
except FileExistsError:
  print('data directory exists, continuing...')
for idx, category in enumerate(categories):
  download_category_from_url(category, urls[idx])
print('All images downloaded into data folder')

data directory exists, continuing...
Creating directory to store dumpling images
Succesfully created dumpling/ directory

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Er

## Generación de datos

In [17]:
# The default version of imgaug doesn't correctly support loading a 1d numpy array to augmenters
!pip install imgaug==0.4.0

     |████████████████████████████████| 952kB 4.5MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.4.0 which is incompatible.
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9


In [0]:
import numpy as np
import imgaug.augmenters as iaa
import cv2
import glob

In [7]:
def augment_data():
  """
  Generates new data based on downloaded images by applying left-to-right flip
  and Gaussian Blur.
  """
  print('Augmenting data by flipping and Gaussian Blur...')
  seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.GaussianBlur(sigma=(0, 3.0))
  ])

  for category in categories:
    batch = []
    for filename in glob.iglob(f'{data_directory}/{category}/*'):
      print(f'Reading {filename}')
      try:
        im = cv2.imread(filename)
        cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
        batch.append(im)
      except Exception as e:
        print(f'Error on image: {filename}, continuing...')
    images_aug = seq(images=np.array(batch))
    for image in images_aug:
      cv2.imwrite(f'{data_directory}/{category}/{uuid.uuid4()}.jpg', image)
      print(f'Artificial data saved for category {category}')
    print(f'=== {len(batch)} new images added to category {category} ===')

augment_data()

Streaming output truncated to the last 5000 lines.
Reading ./drive/My Drive/data/dumpling/3b0f04f8-ac84-4f05-9e4a-760de9464e8e.jpg
Reading ./drive/My Drive/data/dumpling/026d3d19-04a5-42a9-b856-d7f3967e96e9.jpg
Reading ./drive/My Drive/data/dumpling/19bb5ee1-1f61-41c8-ab7a-4a20b5c067cd.jpg
Reading ./drive/My Drive/data/dumpling/44fdfb2e-3d4d-4c6e-ac88-a5c80c1e6351.jpg
Error on image: ./drive/My Drive/data/dumpling/44fdfb2e-3d4d-4c6e-ac88-a5c80c1e6351.jpg, continuing...
Reading ./drive/My Drive/data/dumpling/16ba5da9-94e8-4a30-8d14-2255d9d3225e.jpg
Reading ./drive/My Drive/data/dumpling/cd6a1ef3-7202-4358-ae90-a152a97ec0c3.jpg
Reading ./drive/My Drive/data/dumpling/283c9b75-85d1-427f-b7ae-5dfe65dd188f.jpg
Reading ./drive/My Drive/data/dumpling/9557b85f-87d1-4818-8fe8-c13cb585c8f5.jpg
Reading ./drive/My Drive/data/dumpling/440a91de-6da3-4b23-bc8d-21bec47833a4.jpg
Error on image: ./drive/My Drive/data/dumpling/440a91de-6da3-4b23-bc8d-21bec47833a4.jpg, continuing...
Reading ./drive/My Driv

## Separar train, test y validate
### Separamos 80% train, 10% validate y 10% test para cada clase

In [ ]:
import random
import shutil

In [ ]:
try:
  os.mkdir(f'{data_directory}/train')
except FileExistsError as e:
  print('train directory already exists, continuing...')
try:
  os.mkdir(f'{data_directory}/test')
except FileExistsError as e:
  print('test directory already exists, continuing...')
try:
  os.mkdir(f'{data_directory}/valid')
except FileExistsError as e:
  print('validate directory already exists, continuing...')

In [ ]:
for category in categories:
  try:
    os.mkdir(f'{data_directory}/train/{category}')
    os.mkdir(f'{data_directory}/valid/{category}')
    os.mkdir(f'{data_directory}/test/{category}')
  except FileExistsError as e:
    print(f'{category} directory already exists, continuing...')

  images = glob.glob(f'{data_directory}/{category}/*')
  for i in random.sample(images, int(len(images) * 0.8)):
    shutil.move(i, f'{data_directory}/train/{category}/')

  images = glob.glob(f'{data_directory}/{category}/*') # Get files again, since they were moved
  for i in random.sample(images, int(len(images) * 0.5)):
    shutil.move(i, f'{data_directory}/valid/{category}/')

  images = glob.glob(f'{data_directory}/{category}/*')
  for i in random.sample(images, int(len(images))):
    shutil.move(i, f'{data_directory}/test/{category}/')

  os.remove(f'{data_directory}/{category}')